In [ ]:
# ==========================================
# 1. Data Preparation (DE/DA Part)
# ==========================================
print("Step 1: Loading and Splitting Data...")

# โหลดข้อมูล
X = new_df.drop(columns=['Churn'])
y = new_df['Churn']

# --- KEY CONCEPT: Train / Test Split ---
# เราแบ่ง Test Set แยกออกมาเลย 20% (Hold-out) เอาไว้สอบ Final
# stratify=y คือการบังคับให้สัดส่วน class 0/1 ใน train และ test เท่าๆ กัน (สำคัญมาก!)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train shape: {X_train.shape}")
print(f"Test shape:  {X_test.shape}")
print("-" * 30)

# ==========================================
# 2. Pipeline Construction (DS Part)
# ==========================================
# อาจารย์จะสร้าง Pipeline กลาง เพื่อกัน Data Leakage
# ขั้นตอนคือ: Scale ข้อมูล -> เลือก Feature -> เข้า Model

# เราจะทำ Model Selection โดยการเปรียบเทียบ 2 โมเดล: Logistic Regression และ Random Forest

pipelines = {
    'lr': Pipeline([
        ('scaler', StandardScaler()),                # ปรับ Scale (DE/DS)
        ('selector', SelectKBest(score_func=f_classif)), # Feature Selection (DS)
        ('classifier', LogisticRegression(max_iter=1000)) # Model
    ]),
    'rf': Pipeline([
        ('scaler', StandardScaler()),                # Random Forest ไม่ต้อง Scale ก็ได้แต่ใส่ไว้ไม่เสียหาย
        ('selector', SelectKBest(score_func=f_classif)),
        ('classifier', RandomForestClassifier(random_state=42))
    ])
}

# ==========================================
# 3. Hyperparameter Tuning & Model Selection
# ==========================================
print("Step 2: Training, Tuning & Feature Selection (Cross-Validation)...")

# กำหนด Parameter Grid ที่จะจูน (รวมถึงจำนวน Feature ที่จะเลือกด้วย)
param_grids = {
    'lr': {
        'selector__k': [5, 10, 15, 'all'],           # ลองเลือก feature 5, 10, 15 ตัว หรือทั้งหมด
        'classifier__C': [0.1, 1, 10]                # จูนความเข้มข้นของ Regularization
    },
    'rf': {
        'selector__k': [5, 10, 'all'],
        'classifier__n_estimators': [50, 100],       # จำนวนต้นไม้
        'classifier__max_depth': [None, 10, 20]      # ความลึกของต้นไม้
    }
}

best_models = {}
results = []

# Loop เพื่อแข่งกันระหว่าง Model (Model Selection Logic)
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    
    # ใช้ GridSearchCV เพื่อทำ Cross-Validation (Validation Set อยู่ในนี้แล้ว)
    # cv=5 คือแบ่ง Train data เป็น 5 ส่วน (Train 4, Val 1) วนจนครบ
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[model_name],
        cv=StratifiedKFold(n_splits=5), # ใช้ Stratified เพื่อความชัวร์เรื่อง Class Imbalance
        scoring='accuracy',
        n_jobs=-1 # ใช้ทุก Core ของ CPU
    )
    
    grid_search.fit(X_train, y_train)
    
    best_models[model_name] = grid_search.best_estimator_
    results.append({
        'Model': model_name,
        'Best CV Score': grid_search.best_score_,
        'Best Params': grid_search.best_params_
    })

# ==========================================
# 4. Evaluation (Final Exam)
# ==========================================
print("-" * 30)
print("Step 3: Final Evaluation on Test Set...")

# เปรียบเทียบผลลัพธ์
results_df = pd.DataFrame(results).sort_values(by='Best CV Score', ascending=False)
print("\nValidation Results (from Cross-Validation):")
print(results_df)

# เลือกผู้ชนะ
winner_name = results_df.iloc[0]['Model']
winner_model = best_models[winner_name]

print(f"\n>>> The Winner Model is: {winner_name}")
print(f">>> Best Parameters: {results_df.iloc[0]['Best Params']}")

# วัดผลกับ Test Set (ที่แอบไว้ตั้งแต่ต้น)
y_pred = winner_model.predict(X_test)

print("\nFinal Test Set Report:")
print(classification_report(y_test, y_pred))

# ดูว่า Feature ไหนถูกเลือกบ้าง (ถ้าเป็น SelectKBest)
mask = winner_model.named_steps['selector'].get_support()
selected_features = X.columns[mask]
print(f"Selected Features ({len(selected_features)}): {list(selected_features)}")